In [ ]:
# Required library for the model development
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from random import randrange, uniform
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
# Set the working directory
os.chdir("D:\Edwisor project")

In [ ]:
os.getcwd()

In [ ]:
# Read the csv files into dataframe
train_df=pd.read_csv("Train_data.csv")
test_df=pd.read_csv("Test_data.csv")

In [ ]:
# Command to know number of null values present in each training csv
train_df.info()

In [ ]:
# Command to know number of null values present in each test csv
test_df.info()

In [ ]:
# Command to see the number of unique values in dataframe for object data type
train_df.describe(include=['O'])

In [ ]:
# Command to see the number of unique values in dataframe for object data type
test_df.describe(include=['O'])

In [ ]:
# Missing value analusis
# create a dataframe to check the missing value count
miss_val1=pd.DataFrame(train_df.isnull().sum()).reset_index()
miss_val2=pd.DataFrame(test_df.isnull().sum()).reset_index()

In [ ]:
miss_val2

In [ ]:
train_df['phone number'] = train_df['phone number'].str.replace('-','')
test_df['phone number'] = test_df['phone number'].str.replace('-','')

In [ ]:
train_df['phone number']  = train_df['phone number'].astype('int')
test_df['phone number']  = test_df['phone number'].astype('int')

In [ ]:
test_df.head(5)

In [ ]:
# Box plot o visualize the number of ouliers
num_df=train_df._get_numeric_data()
for i in num_df.columns:
    train_df.boxplot(column=i,by='Churn')

In [ ]:
# Function to convert the object datatype tp categorical variable
def ObjToCat(df):
    lis=[]
    for i in range(0,df.shape[1]):
        if (df.iloc[:,i].dtype=='object'):
            df.iloc[:,i]=pd.Categorical(df.iloc[:,i])
            df.iloc[:,i]=df.iloc[:,i].cat.codes
            df.iloc[:,i]=df.iloc[:,i].round()
            df.iloc[:,i]=df.iloc[:,i].astype('object')
            lis.append(df.columns[i])
    return lis

In [ ]:
lis1= ObjToCat(train_df)
lis2= ObjToCat(test_df)

In [ ]:
num_df=train_df._get_numeric_data()

In [ ]:
type(num_df)

In [ ]:
# Code to plot the histogram for all the numeric variables
for i, col in enumerate(num_df.columns):
    plt.figure(i)
    sns.distplot(num_df[col])

In [ ]:
num_df.columns

In [ ]:
# Feature selection
##### Plot the correlatuion plot for all the numeric data variab;es
f,ax=plt.subplots(figsize=[15,12])
test_corr_data=num_df.corr()
train_corr_data=num_df.corr()
sns.heatmap(test_corr_data,mask=np.zeros_like(test_corr_data,dtype=np.bool),cmap=sns.diverging_palette(220,10,as_cmap=True),
           square=True,ax=ax)

In [ ]:
# perform the chi-square test of independence on object data variable
cat_names=["state","international plan", "voice mail plan"]
for i in cat_names:
    print(i)
    chi2, p, dof, exp=chi2_contingency(pd.crosstab(train_df['Churn'],train_df[i]))
    chi2, p, dof, exp=chi2_contingency(pd.crosstab(test_df['Churn'],train_df[i]))
    print(p)

In [ ]:
#train_df=train_df.drop(['state', 'area code', 'phone number'],axis=1)
test_df=test_df.drop(['state', 'area code', 'phone number','total day minutes',
                               'total eve minutes','total night minutes','total intl minutes'],axis=1)
train_df=train_df.drop(['state', 'area code', 'phone number','total day minutes',
                               'total eve minutes','total night minutes','total intl minutes'],axis=1)

In [ ]:
new_df = train_df.select_dtypes([np.number])

In [ ]:
train_df.dtypes

In [ ]:
# Normalisation
# As most of the data ser are skewed let's perform the normalization
for i in new_df.columns:
    print(i)
    train_df[i]=(train_df[i]-min(train_df[i]))/(max(train_df[i])-min(train_df[i]))
    test_df[i]=(test_df[i]-min(test_df[i]))/(max(test_df[i])-min(test_df[i]))

In [ ]:
# Combine the data frame as motst of the data set are same and also variable to perform the analysis are also same0
cust_df=pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
#replace target categories with Yes or No
cust_df['Churn'] = cust_df['Churn'].replace(0, 'No')
cust_df['Churn'] = cust_df['Churn'].replace(1, 'Yes')

In [ ]:
# Divide the data in train and test
X = cust_df.values[:, 0:13]
Y = cust_df.values[:,13]

X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.2)

In [ ]:
len(X_train), len(X_test)

In [ ]:
# Function to print graphical confusion matrix representation
def conf(algo_name,X_test, y_test):
    y_pred = algo_name.predict(X_test)
    forest_cm = metrics.confusion_matrix(y_pred, y_test)
    sns.heatmap(forest_cm, annot=True, fmt='.2f',xticklabels = ["0", "1"] , yticklabels = ["0", "1"] )
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.title(str(algo_name)[0:str(algo_name).find('(')])

In [ ]:
# Function to print the accuracy and False negative rate 
def confuMatrix(act, pre):
    CM = pd.crosstab(act, pre)
    print(CM)

    #let us save TP, TN, FP, FN
    TN = CM.iloc[0,0]
    FN = CM.iloc[1,0]
    TP = CM.iloc[1,1]
    FP = CM.iloc[0,1]

    #check accuracy of model
    print(((TP+TN)*100)/(TP+TN+FP+FN))

    #False Negative rate 
    print (FN*100/(FN+TP))

In [ ]:
# Model development
# Decision tree Classifier Model development
C50_model = tree.DecisionTreeClassifier(criterion='entropy').fit(X_train, Y_train)
#predict new test cases
C50_Predictions = C50_model.predict(X_test)
print(classification_report(Y_test, C50_Predictions))
confuMatrix(Y_test,C50_Predictions)
conf(C50_model,X_test, Y_test)
# Accuracy 92.6
# FNR 24.82

In [ ]:
# Random Forest Classifier model development
RF_model = RandomForestClassifier(n_estimators = 50).fit(X_train, Y_train)
#predict new test cases
RF_Predictions = RF_model.predict(X_test)
print(classification_report(Y_test, RF_Predictions))
conf(RF_model,X_test, Y_test)
confuMatrix(Y_test,RF_Predictions)
# Accuracy 94
# FNR 26.95

In [ ]:
# Code to print important parameter percentage share
for name, importance in zip(cust_df.columns, RF_model.feature_importances_):
     print(name, "=", importance*100)

In [ ]:
# Logistic regression model.
#replace target categories with Yes or No
cust_df['Churn'] = cust_df['Churn'].replace('No', 0)
cust_df['Churn'] = cust_df['Churn'].replace('Yes', 1)

In [ ]:
#Create logistic data. Save target variable first
cust_df_logit = pd.DataFrame(cust_df['Churn'])
#Add continous variables
cust_df_logit = cust_df_logit.join(cust_df[new_df.columns])
##Create dummies for categorical variables
cat_names=["international plan", "voice mail plan"]
for i in cat_names:
    temp = pd.get_dummies(cust_df[i], prefix = i)
    cust_df_logit = cust_df_logit.join(temp)

In [ ]:
Sample_Index = np.random.rand(len(cust_df_logit)) < 0.8
train = cust_df_logit[Sample_Index]
test = cust_df_logit[~Sample_Index]

In [ ]:
#select column indexes for independent variables
train_cols = train.columns[1:15]
logit = sm.Logit(train['Churn'], train[train_cols]).fit()
logit.summary2()
#Predict test data
test['Actual_prob'] = logit.predict(test[train_cols])
test['ActualVal'] = 1
test.loc[test.Actual_prob < 0.5, 'ActualVal'] = 0
#Build confusion matrix
print(confuMatrix(test['Churn'],test['ActualVal']))
print(classification_report(test['Churn'], test['ActualVal']))
## Accuracy 86.85770750988142
## FNR      73.75886524822695

In [ ]:
forest_cm=metrics.confusion_matrix(test['Churn'],test['ActualVal'])
sns.heatmap(forest_cm, annot=True, fmt='.2f',xticklabels = ["0", "1"] , yticklabels = ["0", "1"] )
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.title(str(logit)[0:str(logit).find('(')])

In [ ]:
## KNN prediction
KNN_model = KNeighborsClassifier(n_neighbors = 3).fit(X_train, Y_train)
# Predict the test cases
KNN_Predictions = KNN_model.predict(X_test)
confuMatrix(Y_test,KNN_Predictions)
print(classification_report(Y_test, KNN_Predictions))
conf(KNN_model,X_test,Y_test)
## Accuracy 88.0
## FNR 65.96

In [ ]:
## Naive Bayes module
from sklearn.naive_bayes import GaussianNB
#Naive Bayes implementation
NB_model = GaussianNB().fit(X_train, Y_train)
# Predict the test cases
NB_Predictions = NB_model.predict(X_test)
confuMatrix(Y_test,NB_Predictions)
print(classification_report(Y_test, NB_Predictions))
conf(NB_model,X_test,Y_test)
## Accuracy 88.9
## FNR 65.96